# Make use of parameterized modules

Timor offers the utility package `parameterized` that allows users to easily generate robots with a fixed topology but changing body parameters.

In [ ]:
import numpy as np
from timor import parameterized, AtomicModule, Body, Connector, Geometry, ModuleAssembly, ModulesDB, ModuleHeader, Transformation
from timor.utilities import spatial, visualization

We start by having a look at a simple parameterized link module.

Adapt `l1` and `l2` to see how the link changes. Be aware that we defined (min, max) limits for the three parameters `r`, `l1` and `l2` to keep the proportions in a realistic range.
We are using SI-standards, so the lengths and radius are given in meters.

In [ ]:
module_header = {'ID': 'L1', 'name': 'Parameterized L link'}
l1 = .4
l2 = .1
r = .05
my_link = parameterized.ParameterizedOrthogonalLink(module_header, lengths=(l1, l2), radius=r, limits=[(0, 0.08), (.1, 2), (.1, 1.8)])
viz = my_link.debug_visualization()
viz.viewer.jupyter_cell(height=350)

## Next, we create a parameterized robot

In order to do so, we need multiple modules and also some joints -- note, how additional joint_parameters (such as joint limits - not to be confused with the dimensionality limits) can be passed along.

In [ ]:
l1 = my_link
l2 = parameterized.ParameterizedCylinderLink({'ID': 'L2', 'name': 'L2'}, length=.5, radius=r, limits=[(0, 0.08), (.3, 2)])
l3 = parameterized.ParameterizedCylinderLink({'ID': 'L3', 'name': 'L3'}, length=.3, radius=r, limits=[(0, 0.08), (.3, 1)])
j1 = parameterized.ParameterizedOrthogonalJoint({'ID': 'J2', 'name': 'J2'}, lengths=(.35, .5), radius=r, limits=[(0, 0.08), (.1, 1), (.1, .8)], joint_parameters={'q_limits': (-1.6, 1.6)})
j2 = parameterized.ParameterizedStraightJoint({'ID': 'J1', 'name': 'J1'}, length=.35, radius=r, limits=[(0, 0.08), (.3, 2)], joint_parameters={'q_limits': (-1.6, 1.6)})

# Parameterized and atomic modules can be mixed (as long as they have matching connectors, but that's necessary for any combination of modules) - let's create a base module as well
base_connectors = (
    Connector('world', spatial.rotX(np.pi), gender='f', connector_type='base'),
    Connector('out', Transformation.from_translation([0, 0, .1 / 2]), gender='m')
)
base_body = Body('base_body', Geometry.EmptyGeometry(), connectors=base_connectors)
base = AtomicModule({'ID': 'B', 'name': 'base'}, bodies=(base_body,))

# We gather all the modules in a database and create an assembly by providing that database and the according IDs
db = ModulesDB({base, l1, l2, l3, j1, j2}, name='parameterized DB')
assembly = ModuleAssembly.from_serial_modules(db, (base.id, l1.id, j1.id, l2.id, j2.id, l3.id))

# As soon as we create a robot from an assembly, we "freeze" the parametric properties:
robot = assembly.robot
robot.update_configuration(np.array((2.1, .1)))
rv = robot.visualize()
rv.viewer.jupyter_cell(height=400)

By "freezing", all parameterized modules are converted to AtomicModules. You can change `l2`, but this won't change the robot we created above.

In [ ]:
frozen = l2.freeze()
print(f"Freezing converts a module of type {type(l2)} to a module of type {type(frozen)}")
print(f"Compare, for example, the masses: {l2.mass:.4f}kg and {frozen.mass:.4f}kg")
l2.resize((r, 0.8))
print(f"After a resize, only the parameterized link changes - the frozen one remains unchanged: {l2.mass:.4f}kg and {frozen.mass:.4f}kg")

# Interactively change the module properties

The benefit of parameterized modules is that their properties can be adapted dynamically. Here, we define a range of possible lengths for the "distal part" of our first link and the total length of bodies of the second joint and visualize how the robot changes. Pay attention to the meshcat window above!

In [ ]:
link_lengths = np.linspace(.35, 1, 50)
joint_lengths = np.linspace(1, .5, 50)

from time import sleep
for len_link, len_joint in zip(link_lengths, joint_lengths):
    l1.resize((l1.radius, len_link, len_link))  # Resize expects parameters for radius, l1, and l2 -- here, we leave the radius unchanged
    j2.resize((j2.radius, len_joint))
    
    # We need to re-instantiate the resulting robot model
    robot = assembly.robot
    robot.update_configuration(np.array((2.1, .1)))
    robot.visualize(rv)
    
    sleep(.1)
    rv.updatePlacements(visualization.VISUAL)